## Data

In [12]:
import torch
import torch.nn as nn
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 14
sequence_length = 6

In [13]:
# Initialize the tokenizer and define a trainer
tokenizer = Tokenizer(WordLevel())
tokenizer.pre_tokenizer = Whitespace()
tokenizer.enable_padding(pad_id=1, 
                         pad_token="<pad>", 
                         length=sequence_length)
tokenizer.enable_truncation(max_length=sequence_length)

# Train the tokenizer on your corpus
trainer = WordLevelTrainer(vocab_size=vocab_size, 
                           special_tokens=["<unk>", "<pad>", "<sos_topic1>", "<sos_topic2>"])
tokenizer.train_from_iterator(corpus, trainer)

In [14]:
corpus[0] = '<sos_topic1> ' + corpus[0]
corpus[1] = '<sos_topic2> ' + corpus[1]

data_x = []
data_y = []
for vector in corpus:
    vector = vector.split()    

    for i in range(1, len(vector)):
        data_x.append( ' '.join(vector[:i]) )
        data_y.append(vector[i])

print(data_x)
print(data_y)

['<sos_topic1>', '<sos_topic1> ăn', '<sos_topic1> ăn quả', '<sos_topic1> ăn quả nhớ', '<sos_topic1> ăn quả nhớ kẻ', '<sos_topic1> ăn quả nhớ kẻ trồng', '<sos_topic2>', '<sos_topic2> có', '<sos_topic2> có chí', '<sos_topic2> có chí thì']
['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', 'có', 'chí', 'thì', 'nên']


In [15]:
# Tokenize and numericalize your samples
def vectorize(x, y, tokenizer, sequence_length):     
    x_ids = tokenizer.encode(x)
    y_ids = tokenizer.token_to_id(y)
    print(x_ids.ids, y_ids)
    return x_ids.ids, y_ids

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, tokenizer, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)

[2, 1, 1, 1, 1, 1] 13
[2, 13, 1, 1, 1, 1] 10
[2, 13, 10, 1, 1, 1] 8
[2, 13, 10, 8, 1, 1] 7
[2, 13, 10, 8, 7, 1] 12
[2, 13, 10, 8, 7, 12] 5
[3, 1, 1, 1, 1, 1] 6
[3, 6, 1, 1, 1, 1] 4
[3, 6, 4, 1, 1, 1] 11
[3, 6, 4, 11, 1, 1] 9


## Train with full data

In [16]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 4)
        self.recurrent = nn.RNN(4, 4, batch_first=True)
        self.linear = nn.Linear(sequence_length*4, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.recurrent(x)
        x = nn.Flatten()(x)
        x = self.linear(x)
        return x

model = TG_Model(vocab_size, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(14, 4)
  (recurrent): RNN(4, 4, batch_first=True)
  (linear): Linear(in_features=24, out_features=14, bias=True)
)


In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [18]:
# train
for _ in range(50):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    #print(loss.item())
    loss.backward()
    optimizer.step()

In [19]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=-1))

tensor([13, 10,  8,  7, 12,  5,  6,  4, 11,  9])


In [20]:
data_y_ids

tensor([13, 10,  8,  7, 12,  5,  6,  4, 11,  9])

## Inference

In [21]:
promt = '<sos_topic2>'
promt_length = 1
promt_ids = tokenizer.encode(promt).ids
print(promt_ids)

[3, 1, 1, 1, 1, 1]


In [22]:
for i in range(sequence_length - promt_length):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    next_id = torch.argmax(outputs, axis=-1)

    promt_ids[promt_length+i] = next_id.item()
    print(promt_ids)

[3, 6, 1, 1, 1, 1]
[3, 6, 4, 1, 1, 1]
[3, 6, 4, 11, 1, 1]
[3, 6, 4, 11, 9, 1]
[3, 6, 4, 11, 9, 9]


In [23]:
print(tokenizer.get_vocab())

{'trồng': 12, 'ăn': 13, 'chí': 4, 'nhớ': 8, '<pad>': 1, '<sos_topic1>': 2, 'kẻ': 7, 'có': 6, 'thì': 11, 'nên': 9, 'quả': 10, '<unk>': 0, 'cây': 5, '<sos_topic2>': 3}
